In [ ]:
###########################################
PROJECT_NAME = 'Airfoil'
###########################################

In [ ]:
!pip install openai==0.28
import openai
import os
!apt-get update
!apt-get install -y elmerfem-csc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.8 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [976 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubunt

In [ ]:
# Install dependencies
!apt-get update
!apt-get install -y git cmake gfortran libblas-dev liblapack-dev libscalapack-mpi-dev libmetis-dev openmpi-bin openmpi-common libopenmpi-dev

# Clone the Elmer repository
!git clone https://github.com/ElmerCSC/elmerfem.git

# Create a build directory and navigate to it
!mkdir elmer-build
%cd elmer-build

# Run cmake and make to build Elmer
!cmake ../elmerfem
!make
!make install

# Add Elmer to the system path
!export PATH=$PATH:/usr/local/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

In [ ]:
openai.api_key =("")

In [ ]:
# Global variables to keep track of the conversation
conversation_history = []
last_response = None


# Compose the system message once and append to history
with open("system_sif.txt", 'r') as file:
    sif_message = file.read().strip()
with open("CFD_airfoil.geo", 'r') as file:
    geometry_message = file.read().strip()

system_message = sif_message + geometry_message

conversation_history.append({"role": "system", "content": system_message})

In [ ]:
#FUNCTIONS FOR INTERACTING WITH CHATBOT

def chat_with_bot(user_messages):
    global last_response
    # Add the new user input to the conversation history
    conversation_history.append({"role": "user", "content": user_messages})
    # Call the OpenAI model
    response = openai.ChatCompletion.create(
        model="gpt-4",  # You can choose the model here (e.g., gpt-4, gpt-3.5-turbo)
        messages=conversation_history,
        temperature=0
    )
    # Extract the model's response
    model_response = response.choices[0].message.content
    # Update the last response
    last_response = model_response
    # Append the model's response to the conversation history
    conversation_history.append({"role": "assistant", "content": model_response})
    # Print the model's response
    return print(model_response)

#Saves .geo file from response
def extract_and_save_sif_file(response_text):
    file_path = f"CFD_airfoil.sif"
    try:
        # Use string formatting explained in system_geo.txt
        start_marker = "//BEGIN_SIF"
        end_marker = f"//END_SIF"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [ ]:
def generate_ELMER_mesh():
    try:
        current_directory = os.getcwd()
        msh_file = os.path.join(current_directory, f'CFD_airfoil.msh')
        print(f"Current directory: {current_directory}")
        !ElmerGrid 14 2 "CFD_airfoil.msh" -autoclean
        print(f"ELMER project created in CFD_airfoil")
        print(f"Copy .sif file in CFD_airfoil")

    except ValueError as e:
        print(f"ERROR.")

In [ ]:
############ INSTRUCTIONS #######################
# Chat with model with chat_with_bot("..."),
# when you are satisfied save the .sif file with
# extract_and_save_sif_file(last_response) and
# generate ELMER project with generate_ELMER_mesh()
# use Paraview for visualization
###############################################

In [ ]:
chat_with_bot("""Create a sif file for a 'flow over an airfoil' simulation with the geometry in the geo file.
""")

Sure, I can help with that. But first, I need some additional information to create the .sif file. 

1. Is the simulation "Steady state" or "Transient"?
2. If it's a steady state simulation, what is the maximum number of iterations you want to set?
3. What are the output intervals for the simulation?
4. What is the convergence tolerance for the steady state?
5. What is the nonlinear system convergence tolerance?
6. What is the maximum number of iterations for the nonlinear system?
7. What is the maximum number of iterations for the linear system?
8. What is the viscosity of the fluid?
9. What is the density of the fluid?
10. What are the initial velocity conditions in the x, y, and z directions?
11. What are the boundary conditions for the inlet, outlet, and walls? This includes the velocity in the x, y, and z directions at the inlet, the pressure at the outlet, and the boundary numbers corresponding to the inlet, outlet, and walls.


In [ ]:
chat_with_bot("""
1. Simulation Type: "Steady state"
2. maximum number of iterations: 700 iterations
3. output intervals: Every 10 iterations
4. convergence tolerance: A convergence tolerance of 1.0e-05
5. nonlinear system convergence tolerance: A convergence tolerance of 1.0e-07
6. maximum number of iterations for the nonlinear system: Maximum of 20 iterations
7. maximum number of iterations for the linear system: Maximum of 500 iterations with a convergence tolerance of 1.0e-10
8. viscosity: 1.81e-5 Pa·s.
9. density:1.225 kg/m³
10. initial velocity: is 50 m/s in the x direction and 0 m/s in the y and z directions.
11. boundary conditions: Inlet: Velocity of 50 m/s in the x-direction, 0 m/s in the y and z directions.Outlet: Pressure of 0 Pascals (gauge pressure).Walls: No-slip condition (velocity at the walls is 0 m/s).
              """)

Thank you for providing the necessary information. Now, I will create the .sif file for your simulation.

//BEGIN_SIF
Header
  CHECK KEYWORDS Warn
  Mesh DB "." "."        
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 31
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 700
  Output Intervals = 10
  Solver Input File = case.sif
  Post File = case.vtu
End

Constants
  Gravity(3) = 0 -9.81 0  ! Gravity in the y-direction for fluid flow
End

Body 1
  Name = "Fluid Domain"
  Equation = 1
  Material = 1
End

Solver 1
  Equation = Navier-Stokes
  Procedure = "FlowSolve" "FlowSolver"
  Variable = "Velocity"
  Variable Dofs = 3
  Exec Solver = Always
  Stabilize = True
  Optimize Bandwidth = True
  Steady State Convergence Tolerance = 1.0e-05
  Nonlinear System Convergence Tolerance = 1.0e-07
  Nonlinear System Max Iterations = 20
  Linear System Solver = Iterative
  Linear System It

In [ ]:
extract_and_save_sif_file(last_response)

CFD_airfoil.sif file saved


In [ ]:
generate_ELMER_mesh()

Current directory: /content/elmer-build

Starting program Elmergrid, compiled on Sep 10 2024
Elmergrid reading in-line arguments
Lower dimensional boundaries will be removed
Materials and boundaries will be renumbered
Nodes that do not appear in any element will be removed
Output will be saved to file CFD_airfoil.

Elmergrid loading data:
-----------------------
Format chosen using the first line: $MeshFormat
Gmsh version is 4.1
Loading mesh in Gmsh format 4.1 from file CFD_airfoil.msh
Reading 37 entities in 0D
Reading 2 entities in 1D
Reading 1 entities in 2D
Allocating for 79 knots and 123 elements.
Leading element dimension is 2
Allocating lookup table for tags of size 37
Defined 37 0DIM entities with geometric tag range [1 37]
Defined 2 1DIM entities with geometric tag range [1 2]
Defined 1 2DIM entities with geometric tag range [1 1]
Physical tag offset for 0D is 0
Geometric tag offset for 0D is 2
Reading 79 nodes in 40 blocks.
Reading 123 elements in 40 blocks.
Moving bulk elemen

In [ ]:
!which ElmerGrid

/usr/local/bin/ElmerGrid


In [ ]:
!pwd

/content/elmer-build


In [ ]:
new_directory = os.path.join(os.getcwd(), f"CFD_airfoil")
os.chdir(new_directory)

In [ ]:
!pwd

/content/elmer-build


In [ ]:
!ElmerSolver CFD_airfoil

ELMER SOLVER (v 9.0) STARTED AT: 2024/09/09 23:42:44
ParCommInit:  Initialize #PEs:            1
MAIN: 
MAIN: =============================================================
MAIN: ElmerSolver finite element software, Welcome!
MAIN: This program is free software licensed under (L)GPL
MAIN: Copyright 1st April 1995 - , CSC - IT Center for Science Ltd.
MAIN: Webpage http://www.csc.fi/elmer, Email elmeradm@csc.fi
MAIN: Version: 9.0 (Rev: 30dfce537, Compiled: 2024-09-09)
MAIN:  Running one task without MPI parallelization.
MAIN:  Running with just one thread per task.
MAIN: =============================================================
MAIN: 
MAIN: 
MAIN: -------------------------------------
MAIN: Reading Model: CFD_Pipe.sif
LoadInputFile: Scanning input file: CFD_Pipe.sif
LoadInputFile: Scanning only size info
LoadInputFile: First time visiting
LoadInputFile: Reading base load of sif file
LoadInputFile: Loading input file: CFD_Pipe.sif
LoadInputFile: Reading base load of sif file
LoadInputFi